In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [3]:
conn.execute("DROP TABLE IF EXISTS especies_nativas_endemicas")
conn.execute("""
    CREATE TABLE especies_nativas_endemicas AS
    SELECT * FROM especies_nativas
    UNION ALL
    SELECT * FROM especies_endemicas
""")

In [4]:
conn.execute("DROP TABLE IF EXISTS gbif_nativas_endemicas")
conn.execute("""
    CREATE TABLE gbif_nativas_endemicas AS
    SELECT *
    FROM gbif
    WHERE species IN (
        SELECT DISTINCT(scientificName_y)
        FROM especies_nativas_endemicas
    )
""").df()

,Count
0,23562


In [5]:
conn.execute("""
    UPDATE gbif_nativas_endemicas
    SET namePublishedInYear = -1
    WHERE namePublishedInYear IS NULL
""").df()

,Count
0,23547


In [6]:
conn.execute("""ALTER TABLE gbif_nativas_endemicas ALTER COLUMN namePublishedInYear SET DATA TYPE INTEGER; """).df()

,Success


In [7]:
sql="""
SELECT
    g.species,

    -- red list: pega apenas 1 valor (ou null)
    FIRST(e.redListCategory) AS redListCategory,

    CASE 
        WHEN MAX(g.year) < 1989 THEN 0
        ELSE 1
    END AS ultimo_ano_coleta,

    CASE                   
        WHEN MAX(g.namePublishedInYear) < 1989 THEN 0
        ELSE 1
    END AS ano_descricao,

    date_diff(
        'day',
        MIN(MAKE_DATE(
            CAST(COALESCE(g.year, 1) AS INTEGER),
            CAST(COALESCE(g.month, 1) AS INTEGER),
            CAST(COALESCE(g.day, 1) AS INTEGER)
        )),
        MAX(MAKE_DATE(
            CAST(COALESCE(g.year, 1) AS INTEGER),
            CAST(COALESCE(g.month, 1) AS INTEGER),
            CAST(COALESCE(g.day, 1) AS INTEGER)
        ))
    ) AS diferenca_data_coleta,

    CASE 
        WHEN COUNT(DISTINCT e.land_regions) > 1 THEN 0
        ELSE 1
    END AS endemica,

    COUNT(g.species) AS total_ocorrencias,

    (SELECT COUNT(gc.taxonomicStatus) FROM gbif_classifications gc WHERE gc.species = g.species AND gc.taxonomicStatus = 'SYNONYM') AS total_sinonimos
FROM gbif_nativas_endemicas g
LEFT JOIN especies_nativas_endemicas e
    ON e.scientificName_y = g.species

GROUP BY g.species;
"""
df = conn.execute(sql).df()

In [8]:
df["enumRedListCategory"] = df["redListCategory"].map({
    "Least Concern": 0,
    "Near Threatened": 0,
    "Critically Endangered": 1,
    "Endangered": 1,
    "Vulnerable": 1
})

In [9]:
df.to_csv("clf.csv", sep=";", quoting=2, index=False, encoding="utf-8")

In [10]:
conn.close()